Quote Tracker (market symbols etc) - A program which can go out and check the current value of stocks for a list of symbols entered by the user. The user can set how often the stocks are checked. For CLI, show whether the stock has moved up or down. Optional: If GUI, the program can show green up and red down arrows to show which direction the stock value has moved.

In [4]:
import yfinance as yf
import time
import sys
from datetime import datetime

def get_stock_data(symbols):
    """
    Fetches current market data for a list of stock symbols 
    
    Input : symbols (list) : A list of string (e.g. ['APPL','MSFT'])

    Return : dictionnary with keys = symbols and values are the feteched info with the value (e.g. {'AAPL' : {'regularMarketPrice': 150.00}})
    
    """

    data = {}
    if not symbols:
        return data
    
    try :
        tickers = yf.Tickers(' '.join(symbols))

        for symbol in symbols:
            symbol_upper = symbol.upper() #always uppercase
            ticker_info = tickers.tickers.get(symbol_upper)

            # Check if data retrieved has a price
            if ticker_info and hasattr(ticker_info, 'info') and 'regularMarketPrice' in ticker_info.info:
                #Check if the price is defined correctly
                if ticker_info.info['regularMarketPrice'] is not None:
                    data[symbol_upper] = ticker_info.info #Store all info
                else:
                    print(f"Price data currently unavailable for {symbol_upper}")
                    data[symbol_upper] = None
            else:
                print(f"Could not fetch data for symbol {symbol_upper}. Make sure it is valid")
                data[symbol_upper] = None
    except Exception as e:
        print(f"An error occured during data fetch {e}")
        #Retrieve data fetched before error
        for symbol in symbols:
            if symbol.upper() not in data:
                data[symbol.upper()] = None
    return data

def display_quotes(current_data, previous_data):
    """
    Display the stock quotes and indicates movement compared to previous data

    Input : current_data(dict) and previous_data(dict)
    """

    print("-"*60)
    print(f"Quotes at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("-"*60)
    print(f"{'Symbol':<10} {'Current Price':<15} {'Change':<10} {'Prev Price':<15}")
    print(f"{'------':<10} {'-------------':<15} {'------':<10} {'----------':<15}")

    # Sort symbol for consistent display
    sorted_symbols = sorted(current_data.keys())

    for symbol in sorted_symbols:
        current_info = current_data.get(symbol)
        prev_info = previous_data.get(symbol)

        if current_info and 'regularMarketPrice' in current_info:
            current_price = current_info['regularMarketPrice']
            price_str = f"{current_price:.2f}"

            prev_price = None
            if prev_info and 'regularMarketPrice' in prev_info:
                prev_price = prev_info['regularMarketPrice']

            change_indicator = " --"
            prev_price_str = "N/A"

            if prev_price is not None:
                prev_price_str = f"{prev_price:.2f}"
                if current_price > prev_price:
                    change_indicator = "⬆ UP"
                elif current_price < prev_price:
                    change_indicator = "⬇ DOWN"
                else:
                    change_indicator = " --"
                
            print(f"{symbol:<10} {price_str:<15} {change_indicator:<10} {prev_price_str:<15}")
        else:
            #If data coudn't be fetched
            print(f"{symbol:<10} {'N/A':<15} {'Error':<10} {'N/A':<15}")
    print("-" * 60)

def main():
    """
    Gets user input and run tracking loop
    """

    symbols_input = input("Enter stock symbols separated by spaces (e.g., AAPL MSFT GOOG): ")
    symbols_list = [s.strip().upper() for s in symbols_input.split() if s.strip()]

    if not symbols_list:
        print("No symbols enterred. Exit")
        sys.exit()

    while True:
        try:
            interval_input = input('Enter refresh interval in seconds')
            refresh_interval = int(interval_input)
            if refresh_interval > 0:
                break
            else:
                print("Interval must be positive")
        except ValueError:
            print("Invalid input. Pleaser enter an integer for the interval")

    #Tracking loop
    previous_stock_data = {}
    print(f"\nStarting tracker for: {', '.join(symbols_list)}")
    print(f"Checking every {refresh_interval} seconds. Press Ctrl+C to stop")

    try:
        while True:
            #Fetch current data
            current_stock_data = get_stock_data(symbols_list)

            #Compare
            display_quotes(current_stock_data, previous_stock_data)

            #Update previous data for the next iteration if succesfully fetched
            for symbol, data in current_stock_data.items():
                if data is not None:
                    previous_stock_data[symbol] = data

            time.sleep(refresh_interval) #Wait for update

    except KeyboardInterrupt:
        print("\nTracker stopped manually")
    except Exception as e:
        print(f"\nAn error occured {e}")
    finally:
        print("Exiting program")

if __name__ == "__main__":
    main()


Starting tracker for: AAPL
Checking every 20 seconds. Press Ctrl+C to stop
------------------------------------------------------------
Quotes at: 2025-04-23 09:11:56
------------------------------------------------------------
Symbol     Current Price   Change     Prev Price     
------     -------------   ------     ----------     
AAPL       199.74           --        N/A            
------------------------------------------------------------
------------------------------------------------------------
Quotes at: 2025-04-23 09:12:16
------------------------------------------------------------
Symbol     Current Price   Change     Prev Price     
------     -------------   ------     ----------     
AAPL       199.74           --        199.74         
------------------------------------------------------------

Tracker stopped manually
Exiting program
